In [14]:
!pip freeze | grep scikit-learn

scikit-learn==1.0.2


In [15]:
import pickle
import pandas as pd

In [16]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [17]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [26]:
# df = read_data('https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_????-??.parquet')
year = 2021
month = 2
df = read_data(f'data/fhv_tripdata_{year:04d}-{month:02d}.parquet')
# for month in range(2,12):
#   new_df = read_data(f'data/fhv_tripdata_{year:04d}-{month:02d}.parquet')
#   df.append(df)
  

In [27]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [28]:
y_pred.mean()

16.191691679979066

In [29]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [30]:
df.head(2)

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration,ride_id
1,B00021,2021-02-01 00:55:40,2021-02-01 01:06:20,173,82,None,B00021,10.666667,2021/02_1
2,B00021,2021-02-01 00:14:03,2021-02-01 00:28:37,173,56,None,B00021,14.566667,2021/02_2


In [32]:
df_pred = pd.DataFrame()
df_pred['ride_id'] = df['ride_id']
df_pred['prediction'] = y_pred
df_pred

,ride_id,prediction
1,2021/02_1,14.539865
2,2021/02_2,13.740422
3,2021/02_3,15.593339
4,2021/02_4,15.188118
5,2021/02_5,13.817206
...,...,...
1037687,2021/02_1037687,12.433246
1037688,2021/02_1037688,12.910885
1037689,2021/02_1037689,15.834923
1037690,2021/02_1037690,16.783176


In [34]:
output_file = f"output/fhv_tripdata_{year:04d}-{month:02d}.parquet"
df_pred.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)
